# Matrix product state simulator example

## Import relevant modules

In [ ]:
from basic_qc_simulator import Circuit
from basic_qc_simulator.simulators import MatrixProductStateSimulator

## Create a circuit

In [ ]:
c = Circuit(3)
c.h(0)
c.cx(0, 1)
c.cx(1, 2)
c.save_result("state_vector")
c.cx(1, 2)
c.cx(0, 1)
c.h(0)
c.save_result("state_vector")
c.draw("mpl")

## Run the matrix product state simulator

In [ ]:
sim = MatrixProductStateSimulator()
sim.run(c)

In [ ]:
sim.results[0].result

In [ ]:
sim.results[1].result

In [ ]:
from basic_qc_simulator import Circuit
from basic_qc_simulator.simulators import MatrixProductStateSimulator
import numpy as np
import logging

logger = logging.getLogger("basic_qc_simulator")
logger.setLevel(logging.DEBUG)
if not logger.hasHandlers():
    logger.addHandler(logging.StreamHandler())

circ = Circuit(3)

circ.h(0)
circ.save_result("matrix_product_state")

sim = MatrixProductStateSimulator()
sv = np.array([1, 0, 0, 0, 0, 0, 0, 0], dtype=np.complex128)
# sv = np.array([1, 0, 0, 0, 0, 0, 0, 1], dtype=np.complex128) / np.sqrt(2)

# sim.run(circ)
# sim.results[0].resultsin

# sim._state_vector_to_left_canonical_matrix_product_state(sv, truncate=True)
sim._state_vector_to_vidal_matrix_product_state(sv, truncate=True)

In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

qc = QuantumCircuit(3)

qc.save_matrix_product_state()
qsim = AerSimulator(method="matrix_product_state")
qsim.run(qc).result().data(0)["matrix_product_state"]

In [ ]:
obj_array = np.array(
    [
        [["a", "b", "c", "d"], ["e", "f", "g", "h"]],
        [["i", "j", "k", "l"], ["m", "n", "o", "p"]],
    ],
    dtype=object,
)

np.tensordot([[1, 2], [3, 4]], obj_array, axes=1)

In [1]:
from basic_qc_simulator.gates import GATETYPES_TO_GATE
from basic_qc_simulator import Circuit


def generate_random_circuit(num_qubits: int, depth: int, seed: int) -> Circuit:
    """
    Generate a random quantum circuit

    Args:
        num_qubits (int): number of qubits in the circuit
        depth (int): depth of the circuit

    Returns:
        Circuit: random quantum circuit
    """
    rng = np.random.default_rng(seed)
    circuit = Circuit(num_qubits)
    for _ in range(depth):
        # one-qubit gates
        for qubit in range(num_qubits):
            gate_type = rng.choice(
                ["i", "x", "y", "z", "h", "s", "t", "phase", "rx", "ry", "rz"]
            )
            if gate_type in ["phase", "rx", "ry", "rz"]:
                circuit.add_gate(
                    GATETYPES_TO_GATE[gate_type](rng.uniform(0, 2 * np.pi)),
                    [qubit],
                )
            else:
                circuit.add_gate(GATETYPES_TO_GATE[gate_type](), [qubit])
        # two-qubit gate
        gate = GATETYPES_TO_GATE[rng.choice(["cx", "ccx", "swap"])]()
        if num_qubits < gate.num_qubits:
            continue
        qubit = rng.integers(num_qubits - gate.num_qubits + 1)
        circuit.add_gate(gate, [qubit + i for i in range(gate.num_qubits)])
    return circuit

In [2]:
from basic_qc_simulator import Circuit
from basic_qc_simulator.simulators import (
    MatrixProductStateSimulator,
    StateVectorSimulator,
)
from basic_qc_simulator.quantum_info.states import MatrixProductState, StateVector
import numpy as np
import logging

# logger = logging.getLogger("basic_qc_simulator")
# logger.setLevel(logging.DEBUG)
# if not logger.hasHandlers():
#     logger.addHandler(logging.StreamHandler())

circ = generate_random_circuit(4, 4, 0)

circ.save_result("state_vector")
# display(circ.draw("mpl"))

sim = StateVectorSimulator()
sim.run(circ)
sv = sim.results[0].result

# sv_init = np.array([1, 0, 0, 0, 0, 0, 0, 0], dtype=np.complex128)
# sv_init = StateVector(sv_init)
# mps = MatrixProductState.from_state_vector(sv_init, truncate=True)
# print(mps)
# print(mps.to_state_vector())

print(sv)
# print(np.abs(sv) ** 2)
mps = MatrixProductState.from_state_vector(sv, truncate=False)
print(mps)
print(mps.to_state_vector())

StateVector([ 4.62123112e-03-2.49878590e-03j  2.49878590e-03+4.62123112e-03j
  4.89413782e-04-2.64635164e-04j  2.64635164e-04+4.89413782e-04j
  2.90439504e-01+5.37136085e-01j -5.37136085e-01+2.90439504e-01j
  3.07591402e-02+5.68856644e-02j -5.68856644e-02+3.07591402e-02j
 -1.03599156e-01+3.32888199e-01j -3.32888199e-01-1.03599156e-01j
 -1.09717202e-02+3.52546904e-02j -3.52546904e-02-1.09717202e-02j
  2.86399173e-03+8.91311642e-04j -8.91311642e-04+2.86399173e-03j
  3.03312469e-04+9.43948031e-05j -9.43948031e-05+3.03312469e-04j])
MatrixProductState(gammas=[
	[[[1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]]],
	[[[-0.00756768+0.00409199j -0.38532846-0.19624292j
   -0.91757322+0.52830793j  0.03654989+0.12856302j]
  [ 0.29714431-0.95479382j  0.1990971 +1.61136688j
   -0.58900798-0.28195471j -0.02715941+0.06678792j]]

 [[-0.47562068-0.87960841j -0.42286845-0.31725035j
    0.20447166-0.06597608j -0.00286557-0.02040433j]
  [-0.00821453-0.00255647j  0.01429588+0.02017839j
    0.16346661+0.18410903j -1.745

TODO: add explanation of MPS state

In [ ]:
from quimb.tensor.tensor_1d import MatrixProductState

print(sv)
for tn in MatrixProductState.from_dense(sv.state_vector, absorb="both").tensors:
    print(tn.data)